In [15]:
import numba
import numpy as np
from numba import jit
from javelin.io import read_stru
from javelin.grid import Grid

In [24]:
grid=Grid()


pzn = read_stru('parts/check/pzn.stru')

qx1, qy1, qz1 = grid.get_squashed_q_meshgrid()
qx2, qy2, qz2 = grid.get_q_meshgrid()

qx3 = qx2*1j
qy3 = qy2*1j
qz3 = qz2*1j

positions = pzn.get_scaled_positions()[:1000]


Found a = 4.06, b = 4.06, c = 4.06, alpha = 90.0, beta = 90.0, gamma = 90.0
Read in these atoms:
O     375000
Pb    125000
Nb     83679
Zn     41321
Name: symbol, dtype: int64


In [25]:
print(numba.typeof(positions))
print(numba.typeof(qx1))
print(numba.typeof(qx3))

array(float64, 2d, A)
array(float64, 3d, C)
array(complex128, 3d, C)


In [26]:
def f1(positions, temp_array, qx, qy, qz):
    for atom in positions:
        dotx = np.exp(qx*atom[0]*1j)
        doty = np.exp(qy*atom[1]*1j)
        dotz = np.exp(qz*atom[2]*1j)
        temp_array += dotx * doty * dotz

In [27]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
print(numba.typeof(temp_array))
%timeit f1(positions, temp_array, qx1, qy1, qz1)
print(temp_array.sum())

array(complex128, 3d, C)
151 ms ± 3.89 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
(339012649.203+463700871.401j)


In [5]:
def f2(positions, temp_array, qx, qy, qz):
    for atom in positions:
        dot = qx*atom[0] + qy*atom[1] + qz*atom[2]
        temp_array += np.exp(dot*1j)

In [6]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
%timeit f2(positions, temp_array, qx2, qy2, qz2)
print(temp_array.sum())

700 ms ± 629 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
(33482730.7855+45797616.9285j)


In [18]:
def f3(positions, temp_array, qx, qy, qz):
    for atom in positions:
        dot = qx*atom[0] + qy*atom[1] + qz*atom[2]
        temp_array += np.exp(dot)

In [19]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
%timeit f3(positions, temp_array, qx3, qy3, qz3)
print(temp_array.sum())

743 ms ± 5.09 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(33482730.7855+45797616.9285j)


In [11]:
@jit
def f1j(positions, temp_array, qx, qy, qz):
    for atom in positions:
        dotx = np.exp(qx*atom[0]*1j)
        doty = np.exp(qy*atom[1]*1j)
        dotz = np.exp(qz*atom[2]*1j)
        temp_array += dotx * doty * dotz

In [12]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
%timeit f1j(positions, temp_array, qx1, qy1, qz1)
print(temp_array.sum())

145 ms ± 211 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
(33482730.7855+45797616.9285j)


In [32]:
f1j.inspect_types()

f1j (array(float64, 2d, A), array(complex128, 3d, C), array(float64, 3d, C), array(float64, 3d, C), array(float64, 3d, C))
--------------------------------------------------------------------------------
# File: <ipython-input-30-dc5a6c3aa231>
# --- LINE 1 --- 
# label 0

@jit()

# --- LINE 2 --- 

def f1j(positions, temp_array, qx, qy, qz):

    # --- LINE 3 --- 
    #   positions = arg(0, name=positions)  :: pyobject
    #   temp_array = arg(1, name=temp_array)  :: pyobject
    #   qx = arg(2, name=qx)  :: pyobject
    #   qy = arg(3, name=qy)  :: pyobject
    #   qz = arg(4, name=qz)  :: pyobject
    #   jump 2
    # label 2
    #   $58 = const(LiftedLoop, LiftedLoop(<function f1j at 0x7fb661b09510>))  :: XXX Lifted Loop XXX
    #   $59 = call $58(positions, qx, qy, qz, temp_array, func=$58, args=[Var(positions, <ipython-input-30-dc5a6c3aa231> (3)), Var(qx, <ipython-input-30-dc5a6c3aa231> (3)), Var(qy, <ipython-input-30-dc5a6c3aa231> (3)), Var(qz, <ipython-input-30-dc5a6c3aa231> (3)

In [13]:
@jit
def f2j(positions, temp_array, qx, qy, qz):
    for atom in positions:
        dot = qx*atom[0] + qy*atom[1] + qz*atom[2]
        temp_array += np.exp(dot*1j)

In [15]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
%timeit f2j(positions, temp_array, qx2, qy2, qz2)
print(temp_array.sum())

702 ms ± 640 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
(33482730.7855+45797616.9285j)


In [13]:
@jit(nopython=True)
def f2j2(positions, temp_array, qx, qy, qz):
    N = positions.shape[0]
    A, B, C = temp_array.shape
    for n in range(N):
        for i in range(A):
            for j in range(B):
                for k in range(C):
                    temp_array += np.exp(qx[i,j,k]*positions[n,0] + qy[i,j,k]*positions[n,1] + qz[i,j,k]*positions[n,2])


In [10]:
temp_array = np.zeros(grid.bins, dtype=np.complex)
%timeit f2j2(positions, temp_array, qx3, qy3, qz3)
print(temp_array.sum())

3.49 s ± 1.65 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(7632659425.35+2460363625.42j)


In [11]:
f2j2.inspect_types()

f2j2 (array(float64, 2d, A), array(complex128, 3d, C), array(complex128, 3d, C), array(complex128, 3d, C), array(complex128, 3d, C))
--------------------------------------------------------------------------------
# File: <ipython-input-9-8d30d923554e>
# --- LINE 1 --- 
# label 0
#   del $const0.3
#   del $0.2
#   del $0.6
#   del $0.10
#   del $24.1
#   del $0.4
#   del $24.3

@jit(nopython=True,parallel=True)

# --- LINE 2 --- 

def f2j2(positions, temp_array, qx, qy, qz):

    # --- LINE 3 --- 
    #   positions = arg(0, name=positions)  :: array(float64, 2d, A)
    #   temp_array = arg(1, name=temp_array)  :: array(complex128, 3d, C)
    #   qx = arg(2, name=qx)  :: array(complex128, 3d, C)
    #   qy = arg(3, name=qy)  :: array(complex128, 3d, C)
    #   qz = arg(4, name=qz)  :: array(complex128, 3d, C)
    #   $0.2 = getattr(value=positions, attr=shape)  :: (int64 x 2)
    #   $const0.3 = const(int, 0)  :: int64
    #   $0.4 = static_getitem(value=$0.2, index=0, index_var=$const0